<a href="https://colab.research.google.com/github/pabvald/Vision-Project-Image-Segmentation/blob/pablo-task1/Vision_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

In [52]:
!pip install scipy==1.1.0

### 1.1 Loading the dataset

In [39]:
import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
from torchvision import models

In [40]:
%config Completer.use_jedi = False
torch.manual_seed(0)
np.random.seed(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  """Entry point for launching an IPython kernel.


#### pascalVOCDataset

In [71]:
class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

#### Factory methods

In [56]:
def segnet_encoder_layer(in_channels, out_channels, kernel_size=3, 
                         stride=1, padding=1): 
  """Create a SegNet encoder layer: Convolution + Batch Normalization + ReLU.

    Args:
      in_channels (int): number of input channels
      out_channels (int): number of output channels
      kernel_size (int): kernel size of the  convolution
      stride (int): stride of the convolution
      padding (int): padding of the convolution

    Returns: 
      (torch.nn.Sequential): SegNet encoder layer
  """
  layer = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
  )
  return layer 

def segnet_encoder_block(in_channels, out_channels):
  """Generate a SegNet encoder block ommitting the final Max Pooling.
  """
  assert len(in_channels) == len(out_channels)
  layers = [] 
  for i in range(len(in_channels)): 
    layer = segnet_encoder_layer(in_channels[i], out_channels[i])
    layers.append(layer)
  block = nn.Sequential(*layers)
  return block

def segnet_decoder_layer(in_channels, out_channels, kernel_size=3, 
                         stride=1, padding=1):
  """Create a SegNet decoder layer: Deconvolution + Batch Normalization + ReLU.

    Args:
      in_channels (int): number of input channels
      out_channels (int): number of output channels
      kernel_size (int): kernel size of the transpose convolution
      stride (int): stride of the transpose convolution
      padding (int): padding of the transpose convolution

    Returns:
      (torch.nn.Sequential): SegNet decoder layer
  """
  layer = nn.Sequential(
      nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, 
                stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
  )
  return layer 

def segnet_decoder_block(in_channels, out_channels):
  """Generate a SegNet decoder block ommitting the initial Upsampling.
  """
  assert len(in_channels) == len(out_channels)
  layers = [] 
  for i in range(len(in_channels)): 
    layer = segnet_decoder_layer(in_channels[i], out_channels[i])
    layers.append(layer)
  block = nn.Sequential(*layers)
  return block

#### SegNet class

In [57]:
class Segnet(nn.Module):
  """SegNet Class"""

  def __init__(self, input_channels, output_channels, vgg16_bn):
    """Initialize an instance of SegNet

      Args:
        input_channels (int): number of input channels 
        output_channels (int): number of output channels
        vgg16_bn (torch.model): pretrained VGG-16 (with Batch Normalization) model
    """
    super(Segnet, self).__init__()
    self.input_channels = input_channels 
    self.output_channels = output_channels    

    # Encoder (VGG16 without Classifier)
    self.enc_block00 = segnet_encoder_block(self.encoder_dims('block00','in'), self.encoder_dims('block00','out'))
    self.enc_block01 = segnet_encoder_block(self.encoder_dims('block01','in'), self.encoder_dims('block01','out'))
    self.enc_block02 = segnet_encoder_block(self.encoder_dims('block02','in'), self.encoder_dims('block02','out'))
    self.enc_block03 = segnet_encoder_block(self.encoder_dims('block03','in'), self.encoder_dims('block03','out'))
    self.enc_block04 = segnet_encoder_block(self.encoder_dims('block04','in'), self.encoder_dims('block04','out'))
    
    # Set pretrained weights 
    self._load_encoder_weights(vgg16_bn)

    # Decoder 
    self.dec_block04 = segnet_decoder_block(self.decoder_dims('block04','in'), self.decoder_dims('block04','out'))
    self.dec_block03 = segnet_decoder_block(self.decoder_dims('block03','in'), self.decoder_dims('block03','out'))
    self.dec_block02 = segnet_decoder_block(self.decoder_dims('block02','in'), self.decoder_dims('block02','out'))
    self.dec_block01 = segnet_decoder_block(self.decoder_dims('block01','in'), self.decoder_dims('block01','out'))
    self.dec_block00 = segnet_decoder_block(self.decoder_dims('block00','in'), self.decoder_dims('block00','out'))

    # Softmax
    self.softmax = nn.Softmax(dim=1)


  def forward(self, x):
    """Compute a forward pass 

      Args: 
        x (torch.Tensor): input image

      Returns: 
        (torch.Tensor, torch.Tensor): output of the network without and applying softmax
    """
    # -- Encoder --
    # Encoder Block 00
    dim00 = x.size()
    enc00 = self.enc_block00(x)
    mp00, indices00 = F.max_pool2d(enc00, kernel_size=2, stride=2, return_indices=True)
    # Encoder Block 01
    dim01 = mp00.size()
    enc01 = self.enc_block01(mp00)
    mp01, indices01 = F.max_pool2d(enc01, kernel_size=2, stride=2, return_indices=True)
    # Encoder Block 02
    dim02 = mp01.size()
    enc02 = self.enc_block02(mp01)
    mp02, indices02 = F.max_pool2d(enc02, kernel_size=2, stride=2, return_indices=True)
    # Encoder Block 03
    dim03 = mp02.size()
    enc03 = self.enc_block02(mp02)
    mp03, indices03 = F.max_pool2d(enc03, kernel_size=2, stride=2, return_indices=True)
    # Encoder Block 04
    dim04 = mp03.size()
    enc04 = self.enc_block02(mp03)
    mp04, indices04 = F.max_pool2d(enc04, kernel_size=2, stride=2, return_indices=True)

    # -- Decoder --
    # Decoder Block 04 
    up04 = F.max_unpool2d(mp04, indices04, kernel_size=2, stride=2, output_size=dim04)
    dec04 = self.dec_block04(up04)
    # Decoder Block 03
    up03 = F.max_unpool2d(dec04, indices03, kernel_size=2, stride=2, output_size=dim03)
    dec03 = self.dec_block03(up03)
    # Decoder Block 02
    up02 = F.max_unpool2d(dec03, indices02, kernel_size=2, stride=2, output_size=dim02)
    dec02 = self.dec_block03(up02)
    # Decoder Block 01
    up01 = F.max_unpool2d(dec02, indices01, kernel_size=2, stride=2, output_size=dim01)
    dec01 = self.dec_block03(up01)
    # Decoder Block 00
    up00 = F.max_unpool2d(dec01, indices00, kernel_size=2, stride=2, output_size=dim00)
    dec00 = self.dec_block03(up00)
    # Softmax
    output = self.softmax(dec00)

    return dec00, output

  def encoder_dims(self, block, io):
    """Obtain the encoder dimensions based on the input dimensions

      Args: 
        block (string): encoder block 
        io (string): 'in' or 'out'

      Returns: 
        (array): dimensions of the corresponding encoder block
    """
    encoder_dimensions = {
      'block00': { 'in': [self.input_channels, 64],'out': [64, 64] },
      'block01': { 'in': [64, 128],       'out': [128, 128] },
      'block02': { 'in': [128, 256, 256], 'out': [256, 256, 256] },
      'block03': { 'in': [256, 512, 512], 'out': [512, 512, 512] },
      'block04': { 'in': [512, 512, 512], 'out': [512, 512, 512] }
    }
    return encoder_dimensions[block][io]

  def decoder_dims(self, block, io): 
    """Obtain the decoder's dimensions based on the output dimensions 
    """
    decoder_dimensions = {
      'block04': { 'in': [512, 512, 512], 'out': [512, 512, 512] },
      'block03': { 'in': [512, 512, 512], 'out': [512, 512, 256] },
      'block02': { 'in': [256, 256, 256], 'out': [256, 256, 128] },
      'block01': { 'in': [128, 128],       'out': [128, 64] },
      'block00': { 'in': [64, 64],         'out': [64, self.output_channels] }
    }
    return decoder_dimensions[block][io]
    
  def _load_encoder_weights(self, vgg16_bn):
    """
      Load the corresponding weights of the train VGG16 model into the encoder.
    """ 
    # Encoder block00
    assert self.enc_block00[0][0].weight.size() == vgg16_bn.features[0].weight.size() 
    assert self.enc_block00[0][0].bias.size() == vgg16_bn.features[0].bias.size() 
    assert self.enc_block00[0][1].weight.size() == vgg16_bn.features[1].weight.size() 
    assert self.enc_block00[0][1].bias.size() == vgg16_bn.features[1].bias.size() 
    assert self.enc_block00[1][0].weight.size() == vgg16_bn.features[3].weight.size() 
    assert self.enc_block00[1][0].bias.size() == vgg16_bn.features[3].bias.size() 
    assert self.enc_block00[1][1].weight.size() == vgg16_bn.features[4].weight.size() 
    assert self.enc_block00[1][1].bias.size() == vgg16_bn.features[4].bias.size() 

    self.enc_block00[0][0].weight.data = vgg16_bn.features[0].weight.data 
    self.enc_block00[0][0].bias.data = vgg16_bn.features[0].bias.data 
    self.enc_block00[0][1].weight.data = vgg16_bn.features[1].weight.data 
    self.enc_block00[0][1].bias.data = vgg16_bn.features[1].bias.data 
    self.enc_block00[1][0].weight.data = vgg16_bn.features[3].weight.data 
    self.enc_block00[1][0].bias.data = vgg16_bn.features[3].bias.data 
    self.enc_block00[1][1].weight.data = vgg16_bn.features[4].weight.data 
    self.enc_block00[1][1].bias.data = vgg16_bn.features[4].bias.data

    # Encoder block01
    assert self.enc_block01[0][0].weight.size() == vgg16_bn.features[7].weight.size() 
    assert self.enc_block01[0][0].bias.size() == vgg16_bn.features[7].bias.size() 
    assert self.enc_block01[0][1].weight.size() == vgg16_bn.features[8].weight.size() 
    assert self.enc_block01[0][1].bias.size() == vgg16_bn.features[8].bias.size() 
    assert self.enc_block01[1][0].weight.size() == vgg16_bn.features[10].weight.size() 
    assert self.enc_block01[1][0].bias.size() == vgg16_bn.features[10].bias.size() 
    assert self.enc_block01[1][1].weight.size() == vgg16_bn.features[11].weight.size() 
    assert self.enc_block01[1][1].bias.size() == vgg16_bn.features[11].bias.size() 
    
    self.enc_block01[0][0].weight.data = vgg16_bn.features[7].weight.data 
    self.enc_block01[0][0].bias.data = vgg16_bn.features[7].bias.data 
    self.enc_block01[0][1].weight.data = vgg16_bn.features[8].weight.data 
    self.enc_block01[0][1].bias.data = vgg16_bn.features[8].bias.data 
    self.enc_block01[1][0].weight.data = vgg16_bn.features[10].weight.data 
    self.enc_block01[1][0].bias.data = vgg16_bn.features[10].bias.data 
    self.enc_block01[1][1].weight.data = vgg16_bn.features[11].weight.data 
    self.enc_block01[1][1].bias.data = vgg16_bn.features[11].bias.data 

    # Encoder block02 
    assert self.enc_block02[0][0].weight.size() == vgg16_bn.features[14].weight.size() 
    assert self.enc_block02[0][0].bias.size() == vgg16_bn.features[14].bias.size() 
    assert self.enc_block02[0][1].weight.size() == vgg16_bn.features[15].weight.size() 
    assert self.enc_block02[0][1].bias.size() == vgg16_bn.features[15].bias.size() 
    assert self.enc_block02[1][0].weight.size() == vgg16_bn.features[17].weight.size() 
    assert self.enc_block02[1][0].bias.size() == vgg16_bn.features[17].bias.size() 
    assert self.enc_block02[1][1].weight.size() == vgg16_bn.features[18].weight.size() 
    assert self.enc_block02[1][1].bias.size() == vgg16_bn.features[18].bias.size() 
    assert self.enc_block02[2][0].weight.size() == vgg16_bn.features[20].weight.size() 
    assert self.enc_block02[2][0].bias.size() == vgg16_bn.features[20].bias.size() 
    assert self.enc_block02[2][1].weight.size() == vgg16_bn.features[21].weight.size() 
    assert self.enc_block02[2][1].bias.size() == vgg16_bn.features[21].bias.size() 

    self.enc_block02[0][0].weight.data = vgg16_bn.features[14].weight.data 
    self.enc_block02[0][0].bias.data = vgg16_bn.features[14].bias.data 
    self.enc_block02[0][1].weight.data = vgg16_bn.features[15].weight.data 
    self.enc_block02[0][1].bias.data = vgg16_bn.features[15].bias.data 
    self.enc_block02[1][0].weight.data = vgg16_bn.features[17].weight.data 
    self.enc_block02[1][0].bias.data = vgg16_bn.features[17].bias.data 
    self.enc_block02[1][1].weight.data = vgg16_bn.features[18].weight.data 
    self.enc_block02[1][1].bias.data = vgg16_bn.features[18].bias.data 
    self.enc_block02[2][0].weight.data = vgg16_bn.features[20].weight.data 
    self.enc_block02[2][0].bias.data = vgg16_bn.features[20].bias.data 
    self.enc_block02[2][1].weight.data = vgg16_bn.features[21].weight.data 
    self.enc_block02[2][1].bias.data = vgg16_bn.features[21].bias.data 

    # Encoder block03
    assert self.enc_block03[0][0].weight.size() == vgg16_bn.features[24].weight.size() 
    assert self.enc_block03[0][0].bias.size() == vgg16_bn.features[24].bias.size() 
    assert self.enc_block03[0][1].weight.size() == vgg16_bn.features[25].weight.size() 
    assert self.enc_block03[0][1].bias.size() == vgg16_bn.features[25].bias.size()
    assert self.enc_block03[1][0].weight.size() == vgg16_bn.features[27].weight.size() 
    assert self.enc_block03[1][0].bias.size() == vgg16_bn.features[27].bias.size() 
    assert self.enc_block03[1][1].weight.size() == vgg16_bn.features[28].weight.size() 
    assert self.enc_block03[1][1].bias.size() == vgg16_bn.features[28].bias.size() 
    assert self.enc_block03[2][0].weight.size() == vgg16_bn.features[30].weight.size() 
    assert self.enc_block03[2][0].bias.size() == vgg16_bn.features[30].bias.size() 
    assert self.enc_block03[2][1].weight.size() == vgg16_bn.features[31].weight.size() 
    assert self.enc_block03[2][1].bias.size() == vgg16_bn.features[31].bias.size() 

    self.enc_block03[0][0].weight.data = vgg16_bn.features[24].weight.data 
    self.enc_block03[0][0].bias.data = vgg16_bn.features[24].bias.data 
    self.enc_block03[0][1].weight.data = vgg16_bn.features[25].weight.data 
    self.enc_block03[0][1].bias.data = vgg16_bn.features[25].bias.data
    self.enc_block03[1][0].weight.data = vgg16_bn.features[27].weight.data 
    self.enc_block03[1][0].bias.data = vgg16_bn.features[27].bias.data 
    self.enc_block03[1][1].weight.data = vgg16_bn.features[28].weight.data 
    self.enc_block03[1][1].bias.data = vgg16_bn.features[28].bias.data 
    self.enc_block03[2][0].weight.data = vgg16_bn.features[30].weight.data 
    self.enc_block03[2][0].bias.data = vgg16_bn.features[30].bias.data 
    self.enc_block03[2][1].weight.data = vgg16_bn.features[31].weight.data 
    self.enc_block03[2][1].bias.data = vgg16_bn.features[31].bias.data 

    # Encoder block04
    assert self.enc_block04[0][0].weight.size() == vgg16_bn.features[34].weight.size() 
    assert self.enc_block04[0][0].bias.size() == vgg16_bn.features[34].bias.size() 
    assert self.enc_block04[0][1].weight.size() == vgg16_bn.features[35].weight.size() 
    assert self.enc_block04[0][1].bias.size() == vgg16_bn.features[35].bias.size() 
    assert self.enc_block04[1][0].weight.size() == vgg16_bn.features[37].weight.size() 
    assert self.enc_block04[1][0].bias.size() == vgg16_bn.features[37].bias.size() 
    assert self.enc_block04[1][1].weight.size() == vgg16_bn.features[38].weight.size() 
    assert self.enc_block04[1][1].bias.size() == vgg16_bn.features[38].bias.size() 
    assert self.enc_block04[2][0].weight.size() == vgg16_bn.features[40].weight.size() 
    assert self.enc_block04[2][0].bias.size() == vgg16_bn.features[40].bias.size() 
    assert self.enc_block04[2][1].weight.size() == vgg16_bn.features[41].weight.size() 
    assert self.enc_block04[2][1].bias.size() == vgg16_bn.features[41].bias.size()

    self.enc_block04[0][0].weight.data = vgg16_bn.features[34].weight.data 
    self.enc_block04[0][0].bias.data = vgg16_bn.features[34].bias.data 
    self.enc_block04[0][1].weight.data = vgg16_bn.features[35].weight.data 
    self.enc_block04[0][1].bias.data = vgg16_bn.features[35].bias.data 
    self.enc_block04[1][0].weight.data = vgg16_bn.features[37].weight.data 
    self.enc_block04[1][0].bias.data = vgg16_bn.features[37].bias.data 
    self.enc_block04[1][1].weight.data = vgg16_bn.features[38].weight.data 
    self.enc_block04[1][1].bias.data = vgg16_bn.features[38].bias.data 
    self.enc_block04[2][0].weight.data = vgg16_bn.features[40].weight.data 
    self.enc_block04[2][0].bias.data = vgg16_bn.features[40].bias.data 
    self.enc_block04[2][1].weight.data = vgg16_bn.features[41].weight.data 
    self.enc_block04[2][1].bias.data = vgg16_bn.features[41].bias.data



In [69]:
local_path = 'VOCdevkit2/VOC2012/' # modify it according to your device
bs = 12
epochs =  1

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [70]:
# dataset variable
training_data = pascalVOCDataset(local_path, split="train", is_transform=True)
validation_data = pascalVOCDataset(local_path, split="val", is_transform=True)

print("Training data instances: ", len(training_data))
print("Validation data instances:", len(validation_data))

FileNotFoundError: ignored

In [ ]:
trainloader = data.DataLoader(training_data, batch_size=bs, shuffle=True, num_workers=0)
valloader = data.DataLoader(validation_data, batch_size=bs, shuffle=True, num_workers=0)

### Load the pretrained VGG16 model (with batch normalization)

In [72]:
#if os.path.isdir('models/vgg16_bn/hub/checkpoints'):
#    checkpoint = torch.load('models/vgg16_bn/hub/checkpoints/vgg16_bn-6c64b313.pth')
#    vgg16_bn = models.vgg16_bn()
#    vgg16_bn.load_state_dict(checkpoint)
#else:
#    os.environ['TORCH_HOME'] = 'models/vgg16_bn'
vgg16_bn = models.vgg16_bn(pretrained=True, progress=True)

KeyboardInterrupt: ignored

In [68]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = Segnet(3, 21, vgg16_bn)

### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [ ]:
# loss function
loss_f = nn.CrossEntropyLoss()

# optimizer variable
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [ ]:
input, labels = next(iter(trainloader))
input.size()

torch.Size([12, 3, 512, 512])

In [ ]:
output = model.forward(input)

In [ ]:
output.size()

In [ ]:
for _ in range(epochs):
  for i, d in enumerate(trainloader):
    # your code goes here

### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [ ]:
def evaluate(ground_truth, predictions):
    
    return f1_score, auc_score, dice_coeeficient

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.